In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
perf = pd.read_csv("Clean_data/Clean_Performance.csv")
conv = pd.read_csv("Clean_data/Clean_conversion.csv")

In [3]:
print(perf.shape, conv.shape)

(7908, 14) (809283, 18)


We merge the conversion file onto the performance file using the ID column. The how tells us how the merge will be performed. For example, an outer join is like a union of two sets where any indexes that aren't in both datasets have their values filled with NaNs. An inner join gives us the intersection of two sets where we join the data only where the IDs match. 

In [4]:
merged_df=conv.merge(perf, on='ID', how='inner')

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768735 entries, 0 to 768734
Data columns (total 31 columns):
Date                                   768735 non-null object
Advertiser ID_x                        768735 non-null int64
Advertiser                             768735 non-null object
Campaign ID_x                          768735 non-null int64
Campaign                               768735 non-null object
Site ID (DCM)_x                        768735 non-null int64
Site (DCM)                             768735 non-null object
Creative ID_x                          768735 non-null object
Creative                               768735 non-null object
Creative Type_x                        768735 non-null object
Placement Pixel Size_x                 768735 non-null object
Platform Type_x                        768735 non-null object
Activity ID                            768735 non-null int64
Activity                               768735 non-null object
Total Conversions        

In [6]:
merged_df_nonulls=merged_df.copy()

Now select the columns to drop as we have lots of duplicates. We drop all the duplicated columns with _y in the their name. 
We update the dataframe merged_df with the removed columns and then rename the columns to remove the _x as it is no longer required. 

In [7]:
cols_to_drop = ['Advertiser ID_y','Campaign ID_y','Site ID (DCM)_y','Creative Type_y', 
                'Placement Pixel Size_y','Platform Type_y','Creative ID_y']

merged_df=merged_df.drop(cols_to_drop, axis=1)
merged_df.rename(columns={"Advertiser ID_x":"Advertiser ID","Campaign ID_x":"Campaign ID",
                          "Site ID (DCM)_x":"Site ID (DCM)","Creative Type_x":"Creative Type",
                          "Placement Pixel Size_x":"Placement Pixel Size","Platform Type_x":"Platform Type",
                          "Creative ID_x":'Creative ID'},inplace=True)


In [8]:
merged_df.columns

Index(['Date', 'Advertiser ID', 'Advertiser', 'Campaign ID', 'Campaign',
       'Site ID (DCM)', 'Site (DCM)', 'Creative ID', 'Creative',
       'Creative Type', 'Placement Pixel Size', 'Platform Type', 'Activity ID',
       'Activity', 'Total Conversions', 'Click-through Conversions',
       'View-through Conversions', 'ID', 'Clicks', 'Impressions',
       'Active View: Viewable Impressions',
       'Active View: Measurable Impressions',
       'Active View: Eligible Impressions', 'Click Rate'],
      dtype='object')

Converting the following columns to strings

In [9]:
merged_df['Advertiser ID']=merged_df['Advertiser ID'].astype(str)
merged_df['Campaign ID']=merged_df['Campaign ID'].astype(str)
merged_df['Activity ID']=merged_df['Activity ID'].astype(str)
merged_df['Site ID (DCM)']=merged_df['Site ID (DCM)'].astype(str)

Create a column for the Activity bins that we'll use in the model

In [10]:
# Duplicate the Activity column as Activity Bin

merged_df['Activity Bin'] = merged_df['Activity']

In [11]:
# Create Activity bins using John's (SMRS) file as a basis, up actually using what we have in the file

act_bins = {
    'Apply Online Click' : 'Application',
    'Apply Now UG Button Click' : 'Application',
    'Apply Now PG Button Click' : 'Application',
    'Apply Online Complete' : 'Application',
    'UG UCAS Application (Exit)' : 'Application',
    'Part-time PDF Application Form' : 'Application',
    'International UG Online Application Form Complete (UWEB)' : 'Application',
    'UCAS Page Exit' : 'Application',
    'PG Application' : 'Application',
    'Launch Space Application' : 'Application',
    
    'Book an Open Day' : 'Open Day',
    'Book Now - Success' : 'Open Day',
    'UG Open Day Complete': 'Open Day',
    'Open Day': 'Open Day',
    'UG Open Day Registration': 'Open Day',
    'PG Open Day Complete': 'Open Day',
    'Part-Time Open Day Complete': 'Open Day',
    'PG Open Day Form': 'Open Day',
    'Book an Open Day Click': 'Open Day',
    'Open Day Landing Page': 'Open Day',
    'PG Open Day Registration': 'Open Day',
    'PG Open Event 22 May 2019': 'Open Day',
    'PG Open Event 24 April 2019': 'Open Day',
    'UG Open Day 16 November 2019': 'Open Day',
    'UG Open Day 2 November 2019': 'Open Day',
    'UG Open Day 8 June 2019': 'Open Day',
    'PG Open Event 17 Apr 2019': 'Open Day',

    
    'Prospectus' : 'Prospectus',
    'Prospectus Request' : 'Prospectus',
    'PG Prospectus Request Complete' : 'Prospectus',
    'Prospectus Request Complete' : 'Prospectus',
    'UG Prospectus Request Complete' : 'Prospectus',
    'UG Course Details Download' : 'Prospectus',
    
    'Course Enquiry - Thank You' : 'Enquiry',
    'International Enquiry' : 'Enquiry',
    'Clearing Enquiry Form Complete' : 'Enquiry',
    'International Enquiry Contact Us Confirmation (Microsite)' : 'Enquiry',
    'ODL Enquiry Complete' : 'Enquiry',
    'BBEC Enquiry Confirmation Page August' : 'Enquiry',
    'Intl Enquiry [UWE] OLD' : 'Enquiry',
    'Business Email' : 'Enquiry',
    'Offline Conversion Tracking - Phone Call' : 'Enquiry',
    'Click to Call' : 'Enquiry',
    'BBEC - Enquire Now Online Survey Click' : 'Enquiry',
    'Professional Development Course Enquiries Button Click' : 'Enquiry',
    'Enquire Now Click (The Forge)' : 'Enquiry',
    'Degree Apprenticeships Mail to Click' : 'Enquiry',
    'Exec Education Mail to Click' : 'Enquiry',
    'Faculty of Arts Creative Industries and Education Mail to' : 'Enquiry',
    'Faculty of Business and Law Mail to' : 'Enquiry',
    'Faculty of Environment and Technology Mail to' : 'Enquiry',
    'Faculty of Health and Applied Sciences Mail to' : 'Enquiry',
    'Exec Education Contact Us Button Click' : 'Enquiry',
    'HDA Enquire Now' : 'Enquiry',
    'International Enquire Now (Microsite)' : 'Enquiry',
    'International Enquire Now (UWEB)' : 'Enquiry',
    'Live Chat Started' : 'Enquiry',
    'Chat Now - International' : 'Enquiry',
    'Cyber Webinar Book' : 'Enquiry',
    
    'Clearing Page' : 'Clearing',
    'Clearing Page [30 Dwell Time]' : 'Clearing',
    'Clearing Landing Page' : 'Clearing',
    'Clearing Find Your Course Click' : 'Clearing',
    
    'PG Homepage' : 'Homepage',
    'Part-time Homepage' : 'Homepage',
    'UG Homepage' : 'Homepage',
    'International Homepage' : 'Homepage',
    
    'Study Pages' : 'Other Pages',
    'SOH School Pages' : 'Other Pages',
    'SSE School Pages' : 'Other Pages',
    'Undergraduate Pages' : 'Other Pages',
    'SOC School Pages' : 'Other Pages',
    'HDA Course Page' : 'Other Pages',
    'UG Scholarship' : 'Other Pages',
    'Part-Time Course' : 'Other Pages',
    'Global Floodlight' : 'Other Pages',
    'Course View' : 'Other Pages',
    
    'UWE International Global AUD REM' : 'Rare',
    'I4G External Site Link Click' : 'Rare',
    
    
}

In [12]:
# Replace Activity Bin with the proper bins from act_bin

merged_df['Activity Bin'].replace(act_bins, inplace=True)

In [13]:
merged_df['Activity Bin'].nunique()

8

Create column for total pixel area

In [14]:
merged_df['Pixel Area'] = merged_df['Placement Pixel Size']

In [15]:
# Create list of unique pixel size values

pix_list = list(merged_df['Placement Pixel Size'].unique())

In [16]:
# Convert pixel size values into an area array saved as a string

area_arr = []
for i,s in enumerate(pix_list):
    splt = s.split()
    area_arr.append(str((int(splt[0])*int(splt[2]))))

In [17]:
# Create conversion transform from pixels to area

area_replace = {'1 x 1':'1',
 '300 x 250' :  '75000',
 '300 x 600' :  '180000',
 '320 x 100' :  '32000',
 '336 x 280' :  '94080',
 '728 x 90' :  '65520',
 '120 x 600' :  '72000',
 '160 x 600' :  '96000',
 '468 x 60' :  '28080',
 '180 x 150' :  '27000',
 '320 x 50' :  '16000',
 '970 x 90' :  '87300',
 '970 x 250' :  '242500',
 '120 x 90' :  '10800',
 '100 x 100' :  '10000',
 '300 x 50' :  '15000',
 '250 x 100' :  '25000', 
 '300 x 380' :  '114000', 
 '200 x 200' :  '40000',
 '250 x 250' :  '62500',
 '300 x 100' :  '30000',
 '600 x 90' :  '54000',
 '300 x 251' :  '75300', 
 '300 x 254' :  '76200', 
 '300 x 258' :  '77400',
 '320 x 250' :  '80000', 
 '160 x 601' :  '96160',
 '300 x 252' :  '75600',
 '300 x 253' :  '75900', 
 '580 x 400' :  '232000',
 '600 x 250' :  '150000', 
 '990 x 100' :  '99000', 
 '640 x 640' :  '409600', 
 '650 x 100' :  '65000',
 '338 x 280' :  '94640',
 '245 x 210' :  '51450'}

In [18]:
# Place correct values i

merged_df['Pixel Area'].replace(area_replace,inplace=True)

Create columns for the month and year

In [19]:
merged_df['Year'] = pd.DatetimeIndex(merged_df['Date']).year
merged_df['Month'] = pd.DatetimeIndex(merged_df['Date']).month

Create column that relates the campaign name from the merged file to the cost file

In [21]:
# Cost file campaign names

cost_camp = np.array(['Clearing 2018', 'UG 2019 Entry', 'Faculties 2019',
       'UCAS Extra 2018', 'UG to PG 2018', 'UCAS Extra 2019', 'BBEC 2018',
       'MSc Cybersecurity 2018', 'MA Virtual Reality 2018',
       'MSc Civil Engineering 2018', 'Adult Nursing 2018',
       'Optometry Nov 2018', 'Urban and Rural Planning 2018',
       'Degree Apprenticeships 2018', 'Education Technology 2018',
       'Vietnam International 2018', 'Thailand International 2018',
       'BSc Computing 2018', 'Science Communications 2018', 'BME 2018',
       'PG August - Sept 2018', 'SOH 2018', 'UG Feb to July 2018',
       'TUBS - Jan - July 2018', 'SOH 2018/19', 'PG September - Jan 2019',
       'TUBS - 18/19', 'SSE - 18/19', 'SCMA 2018', 'SCMA PG 2018',
       'Summer University 2018', 'MSc Crime Intelligence 2018',
       'Animal Science 2018', 'Early Awareness 2018'])

In [95]:
# NEED TO TALK TO JOHN ABOUT THESE AND WHICH ONES CAN BE USED FOR WHAT!!!!

cost_rename = {
    'Clearing 2018 (036580)' : 'Clearing 2018',
    'Clearing 2018 (034954)' : 'Clearing 2018',
    'Clearing (038093)' : 'Clearing 2018',
    
    'UCAS Extra 2018 (036577)' : 'UCAS Extra 2018',
    'UCAS Extra 2019 (038840)' : 'UCAS Extra 2019',
    'UG SSED UCAS Extra 2019 (039374)' : 'UCAS Extra 2019',
    
    'BBEC (035126)' : 'BBEC 2018', 
    'BBEC Phase 2 (035126)' : 'BBEC 2018',
    
    'MA Virtual Reality (038342)' : 'MA Virtual Reality 2018',
    
    'MSc Civil Engineering (035126)' : 'MSc Civil Engineering 2018',
    'MEng Civil and Environmental Engineering (035126)' : 'MSc Civil Engineering 2018',
    
    'BSc Optometry (037407)' : 'Optometry Nov 2018',
    
    'UWE PG2017 MSc Urban and Rural Planning (033135)' : 'Urban and Rural Planning 2018',
    'MS Urban and Rural Planning (143224)' : 'Urban and Rural Planning 2018',
    'Urban and Rural Planning (035126)' : 'Urban and Rural Planning 2018',
    
    'Degree Apprenticeships & Exec Education Campaign (037591)' : 'Degree Apprenticeships 2018',
    'Degree Apprenticeships  (034962)' : 'Degree Apprenticeships 2018',
    
    'UWE International 2017 - Vietnam (033392)' : 'Vietnam International 2018',
    'UWE International - Thailand (033421)' : 'Thailand International 2018',
    
    'BME (35127)' : 'BME 2018',
    
    'UG to PG (033860)' : 'UG to PG 2018',
    'UG to PG 2018 Intake (035126)' : 'UG to PG 2018',
    
    'UG Feb 18 - Jul 18 (034954)' : 'UG Feb to July 2018',

    'TUBS Campaign (036286)' : 'TUBS - Jan - July 2018', 
    'TUBS Clearing  (034951)' : 'TUBS - Jan - July 2018',
    'TUBS 2018/19 (034951)' : 'TUBS - 18/19',
    
    'SSE - PAD 18 (034958)' : 'SSE - 18/19',
    'SSE PAD January (034958)' : 'SSE - 18/19',
    'Teesside - SSE - Design, Fashion & Textiles Campaign - 036315' : 'SSE - 18/19',
    'SSE UCAS Extra (034958)' : 'SSE - 18/19',
    'SSED May/June Campaign ( 037411)' : 'SSE - 18/19',
    'UG SSED UCAS Extra 2019 (039374)' : 'SSE - 18/19',
    
    'SCMA Social Campaign (037727)' : 'SCMA 2018', 
    'SCMA PG (034956)' : 'SCMA PG 2018',
    
    'Summer University (037267)' : 'Summer University 2018',
    'Summer School (145832)' : 'Summer University 2018',

    'Animal Science & Welfare (034958)' : 'Animal Science 2018',
    
    'Early Awareness Year 12 Campaign 2018 (034954)' : 'Early Awareness 2018',
    'UG Early Awareness (039262)' : 'Early Awareness 2018',

}

In [96]:
clear = [x for x in merged_df['Campaign'].unique() if 'Clearing' in x]
ucas = [x for x in merged_df['Campaign'].unique() if 'UCAS' in x]
bbec = [x for x in merged_df['Campaign'].unique() if 'BBEC' in x]
virt = [x for x in merged_df['Campaign'].unique() if 'Virtual' in x]
civil = [x for x in merged_df['Campaign'].unique() if 'Civil' in x]
opt = [x for x in merged_df['Campaign'].unique() if 'Optometry' in x]
urban = [x for x in merged_df['Campaign'].unique() if 'Urban' in x]
deg = [x for x in merged_df['Campaign'].unique() if 'Degree' in x]
inter = [x for x in merged_df['Campaign'].unique() if 'International' in x]
pg = [x for x in merged_df['Campaign'].unique() if 'PG' in x]
soh = [x for x in merged_df['Campaign'].unique() if 'SOH' in x]
ug = [x for x in merged_df['Campaign'].unique() if 'UG' in x]
tubs = [x for x in merged_df['Campaign'].unique() if 'TUBS' in x]
sse = [x for x in merged_df['Campaign'].unique() if 'SSE' in x]
scma = [x for x in merged_df['Campaign'].unique() if 'SCMA' in x]
summer = [x for x in merged_df['Campaign'].unique() if 'Summer' in x]
animal = [x for x in merged_df['Campaign'].unique() if 'Animal' in x]
early = [x for x in merged_df['Campaign'].unique() if 'Early' in x]

In [97]:
early

['Early Awareness Year 12 Campaign 2018 (034954)',
 'UG Early Awareness (039262)']

In [98]:
merged_df['Campaign'].unique()

array(['UWE UG2017 HAS Campaign (032996)',
       'UWE PG2017 MSc Urban and Rural Planning (033135)',
       'Decision 2018  (034386)', 'Thailand 2017/18 Intake (035618)',
       'DART Search', 'Natural Search',
       'UWE UG2017 FET Campaign (032949)',
       'Profile Raising & Open Event (035126)',
       'MS Urban and Rural Planning (143224)',
       'UWE UG 2018 Awareness Campaign (033842)',
       'Vietnam 2017/18 Intake (035616)',
       'UWE International 2017  (033347)',
       'Creative Degree Show 2017 (034328)', 'UG to PG (033860)',
       'UWE International - Nigeria (033422)', 'UCAS Extra 2018 (036577)',
       'UG 2019 Student Intake (036350)',
       'UWE UG2017 - UCAS Extra (033879)',
       'Profile Raising & Open Events 2017/18 (035742)',
       'Creative Degree Show (35123)', 'Launch Space (036993)',
       'UG to PG 2018 Intake (035126)', 'BSc Optometry (037407)',
       'MSc Cyber Security (035126)', 'BBEC (035126)',
       'Clearing 2018 (036580)', 'MSc Civil Eng

In [99]:
merged_df['Campaign Name (Cost File)'] = merged_df['Campaign']

In [100]:
merged_df['Campaign Name (Cost File)'].replace(cost_rename,inplace=True)

In [102]:
merged_df['Campaign Name (Cost File)'].nunique()

99

In [43]:
cond1 = merged_df['Clicks'] >= 0
cond2 = merged_df['Impressions'] == 0
cond3 = merged_df['Placement Pixel Size']=='1 x 1'
cond4 = merged_df['Creative Type']=='Display'
merged_df[cond3]['Creative Type'].value_counts()

(not set)    97928
Tracking     37991
Display      11612
Name: Creative Type, dtype: int64

When the number of Impressions == 0 and Clicks is greater than zero we need a new Creative Type called "one_pix_track". 

In [47]:
#cond1=merged_df['Clicks']>0
#cond2=merged_df['Impressions']==0
#merged_df.loc[cond1 & cond2, 'Creative Type'] = 'one_pix_track'

#cond1=merged_df['Placement Pixel Size']=='1 x 1'
#merged_df.loc[cond1, 'Creative Type'] = 'Tracking'

merged_df.head()

,Date,Advertiser ID,Advertiser,Campaign ID,Campaign,Site ID (DCM),Site (DCM),Creative ID,Creative,Creative Type,...,Clicks,Impressions,Active View: Viewable Impressions,Active View: Measurable Impressions,Active View: Eligible Impressions,Click Rate,Activity Bin,Pixel Area,Year,Month
0,2018-01-01,4476036,University of the West of England (UWE),10379722,UWE UG2017 HAS Campaign (032996),1434033,The Complete University Guide,(not set),(not set),(not set),...,1,0,0,0,0,NaN,Other Pages,1,2018,1
1,2018-01-01,4476036,University of the West of England (UWE),10379722,UWE UG2017 HAS Campaign (032996),1434033,The Complete University Guide,(not set),(not set),(not set),...,1,0,0,0,0,NaN,Other Pages,1,2018,1
2,2018-01-03,4476036,University of the West of England (UWE),10379722,UWE UG2017 HAS Campaign (032996),1434033,The Complete University Guide,(not set),(not set),(not set),...,1,0,0,0,0,NaN,Other Pages,1,2018,1
3,2018-01-03,4476036,University of the West of England (UWE),10379722,UWE UG2017 HAS Campaign (032996),1434033,The Complete University Guide,(not set),(not set),(not set),...,1,0,0,0,0,NaN,Other Pages,1,2018,1
4,2018-01-01,4476036,University of the West of England (UWE),10496824,UWE PG2017 MSc Urban and Rural Planning (033135),1503097,Google Display Network,75480110,Tracking Ad - 1x1,Tracking,...,1,0,0,0,0,NaN,Other Pages,1,2018,1


Write out merged_df to a csv file

In [48]:
merged_df.to_csv("Clean_data/Merged_data.csv", index=False)

Sanity check that the number of NaN entries for Click Rate is what we expect - we probably do not need this though

In [21]:
merged_df['Click Rate'].isnull().sum()

103281

In [22]:
merged_df[merged_df['Creative Type']=='Tracking']['Placement Pixel Size'].value_counts()

1 x 1        37991
300 x 250       28
Name: Placement Pixel Size, dtype: int64

In [23]:
merged_df['Creative Type'].value_counts()

Display      632786
(not set)     97930
Tracking      38019
Name: Creative Type, dtype: int64